In [1]:
# %load_ext jupyter_black
# %load_ext autoreload
# %autoreload 2

In [2]:
import torch
from torch.utils.data import DataLoader
import torchvision
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

from torchinfo import summary

import constants
import sys

In [3]:
# Add the root directory to the Python path
sys.path.append("/workspaces/kg-ylrust/")

# Import the local modules
from src.dataset import CropDataset
from src.checks import single_pass_clf
from src.helper import set_seeds

: 

Simple baseline model
- Resnet18
- no major data augmentations
- sigfigs
- random seeds

In [5]:
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cpu'

In [6]:
# Model definitions
weights = torchvision.models.ResNet18_Weights.DEFAULT
model_rgb = torchvision.models.resnet18(weights=weights).to(device)
auto_transforms = weights.transforms()

Create datasets and datafolders

In [8]:
### Dataset does the following:
# Take a folder of .tif files.
# Class is defined in higher level.
# .tif files are combined into a single rgb image.
train_dataset = CropDataset(
    constants.train_dir,
    transform=auto_transforms,
    bands="rgb",
)
val_dataset = CropDataset(
    constants.validation_dir,
    transform=auto_transforms,
    bands="rgb",
)

# Create data loaders
train_loader = DataLoader(
    train_dataset,
    batch_size=32,
    shuffle=True,
    num_workers=1,
    pin_memory=True,
    persistent_workers=True,
)
val_loader = DataLoader(
    val_dataset,
    batch_size=32,
    shuffle=False,
    num_workers=1,
    pin_memory=True,
    persistent_workers=True,
)

# Quick check
for images, labels in train_loader:
    print(images.shape)  # Should be [batch_size, 3, H, W]
    print(labels)  # Should be a list of crop types
    break

NameError: name 'CropDataset' is not defined

Get ready for Transfer Learning

In [15]:
# Freeze earlier layers
for param in model_rgb.parameters():
    param.requires_grad = False

# Set the manual seeds
set_seeds()

# Get the length of class_names (one output unit for each class)
class_names = set(labels)
output_shape = len(class_names)

# Recreate the classifier layer and seed it to the target device
model_rgb.fc = torch.nn.Linear(
    in_features=model_rgb.fc.in_features,
    out_features=output_shape,  # same number of output units as our number of classes
).to(device)

for param in model_rgb.fc.parameters():
    param.requires_grad = True

In [16]:
summary(
    model=model_rgb,
    input_size=(32, 3, 224, 224),  # make sure this is "input_size", not "input_shape"
    # col_names=["input_size"], # uncomment for smaller output
    col_names=["input_size", "output_size", "num_params", "trainable"],
    col_width=20,
    row_settings=["var_names"],
)

Layer (type (var_name))                  Input Shape          Output Shape         Param #              Trainable
ResNet (ResNet)                          [32, 3, 224, 224]    [32, 4]              --                   Partial
├─Conv2d (conv1)                         [32, 3, 224, 224]    [32, 64, 112, 112]   (9,408)              False
├─BatchNorm2d (bn1)                      [32, 64, 112, 112]   [32, 64, 112, 112]   (128)                False
├─ReLU (relu)                            [32, 64, 112, 112]   [32, 64, 112, 112]   --                   --
├─MaxPool2d (maxpool)                    [32, 64, 112, 112]   [32, 64, 56, 56]     --                   --
├─Sequential (layer1)                    [32, 64, 56, 56]     [32, 64, 56, 56]     --                   False
│    └─BasicBlock (0)                    [32, 64, 56, 56]     [32, 64, 56, 56]     --                   False
│    │    └─Conv2d (conv1)               [32, 64, 56, 56]     [32, 64, 56, 56]     (36,864)             False
│    │    

Check 1: Test Shapes Input & Output
- Forward pass on a single image 

In [17]:
single_pass_clf(dataloader=train_loader, model=model_rgb, device=device)

Single image shape: torch.Size([1, 3, 224, 224])

Output logits:
tensor([[ 0.0777, -0.4661, -0.5256, -0.1546]])

Output prediction probabilities:
tensor([[0.3424, 0.1988, 0.1873, 0.2714]])

Output prediction label:
tensor([0])

Actual label:
maize


: 

In [ ]:
TODO: 
- add save_model function
- add naming parameters

# 8. Create a new loss and optimizer for every model
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(params=model.parameters(), lr=0.001)

# 9. Train target model with target dataloaders and track experiments
train(model=model_rgb,
        train_dataloader=train_dataloader,
        test_dataloader=test_dataloader,
        optimizer=optimizer,
        loss_fn=loss_fn,
        epochs=epochs,
        device=device,
        writer=create_writer(experiment_name='RGB',
                            model_name='Resnet18-ImageNet',
                            extra=f"{epochs}_epochs"))

# 10. Save the model to file so we can get back the best model
save_filepath = f"{model_name}_{dataloader_name}_{epochs}_epochs.pth"
save_model(model=model_rgb,
            target_dir="models",
            model_name=save_filepath)

1) Build base

2) Build EarlyStopper

3) Build StatifiedKFold